## 1. Dataloader

In [3]:
import cv2
import json
import numpy as np
import imgaug.augmenters as iaa

from pathlib import Path
from natsort import natsorted

from data.augmenter import Augmenter

In [4]:
label_paths = natsorted(Path('../dataset/focused_scene_text_2013/train/').glob('*.json'), key=lambda x: x.stem)
image_paths = natsorted(Path('../dataset/focused_scene_text_2013/train/').glob('*.jpg'), key=lambda x: x.stem)

data_pairs = [(image_path, label_path) for image_path, label_path in zip(label_paths, image_paths) if image_path.stem == label_path.stem]

print(len(data_pairs))

229


In [8]:
augmenter = Augmenter()

image_path = str(image_paths[0])
label_path = str(label_paths[0])

image = cv2.imread(image_path)
thickness = max(image.shape) // 400

with open(file=label_path, mode='r', encoding='utf-8') as f:
    label = json.load(f)

image, label = resize(image, label)

for shape in label['shapes']:
    if shape['shape_type'] == 'rectangle':
        points = to_4points(shape['points'])
        points = to_valid_poly(points, image_height=image.shape[0], image_width=image.shape[1])
        cv2.rectangle(
            img=image,
            pt1=(int(points[0][0]), int(points[0][1])),
            pt2=(int(points[1][0]), int(points[1][1])),
            color=(0, 255, 0),
            thickness=thickness
        )
    elif shape['shape_type'] == 'polygon':
        points = to_valid_poly(shape['points'], image_height=image.shape[0], image_width=image.shape[1])
        cv2.polylines(img=image, pts=[np.int32(points)], isClosed=True, color=(0, 255, 0), thickness=thickness)
        print(points)
    else:
        raise ValueError(f"visual function for {shape['shape_type']} is not implemented.")

cv2.imwrite('image.png', image)

[[7.584381103515625, 170.6666717529297], [344.785888671875, 170.6666717529297], [344.785888671875, 241.3333282470703], [7.584381103515625, 241.3333282470703]]
[[387.4359130859375, 170.6666717529297], [464.739013671875, 170.6666717529297], [464.739013671875, 225.3333282470703], [387.4359130859375, 225.3333282470703]]
[[0.0, 266.6666564941406], [280.8109130859375, 266.6666564941406], [280.8109130859375, 324.0], [0.0, 324.0]]
[[322.128173828125, 265.33331298828125], [446.0797119140625, 265.33331298828125], [446.0797119140625, 318.6666564941406], [322.128173828125, 318.6666564941406]]
[[0.0, 361.3333435058594], [306.1343994140625, 361.3333435058594], [306.1343994140625, 416.0], [0.0, 416.0]]


True

In [5]:
def resize(image, label, imsize=640):
    f = imsize / min(image.shape[:2])
    image, label = augmenter.apply(
        image=image, label=label,
        augmenter=iaa.Resize(size=f)
    )
    image, label = augmenter.apply(
        image=image, label=label,
        augmenter=iaa.CropToFixedSize(width=imsize, height=imsize, position='uniform')
    )
    return image, label

def to_valid_poly(polygon, image_height, image_width):
    polygon = np.array(polygon)
    polygon[:, 0] = np.clip(polygon[:, 0], a_min=0, a_max=image_width - 1)  # x coord not max w-1, and not min 0
    polygon[:, 1] = np.clip(polygon[:, 1], a_min=0, a_max=image_height - 1)  # y coord not max h-1, and not min 0
    return polygon.tolist()

def to_4points(points):
    x1, y1 = points[0][0], points[0][1]
    x2, y2 = points[1][0], points[1][1]
    return [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]